Let's import the agent SDK classes and functions we need, as long as sendgrid helper info.

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

### Email Function
We could use this to allow the LLM to email us for a variet of reasons. For example, maybe it wants to tell us there's a question it couldn't answer. Another example might be if someone is reaching out to us and the LLM wants to say something like, "Hey, Jonas from Masifunde is asking to connect."

In [3]:

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jim@masinyusane.org")
    to_email = To("jim@masinyusane.org")
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

UnauthorizedError: HTTP Error 401: Unauthorized

This is some background info for us to dynamically plug into each of the agent contexts

In [4]:
zz_background = """# PROGRAMME OVERVIEW
- Programme: **Zazi iZandi** (South Africa)
- Intervention: Teaching small groups of **7 children** their letter sounds in a **frequency‑based sequence**.
- Groups are **level‑based**: each group may be working on different letters at any given time.
- Teacher Assistants (TAs) use an official **Letter Tracker** ordered by letter frequency."""

Let's create some instructions for our agents. Putting this here to make the code more readable.

In [5]:
instructions_2023 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}"
instructions_2024 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2024. {zz_background}"
instructions_2025 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2025. {zz_background}"


Okay, let's create some agents!

In [6]:
zazi_2023_agent = Agent(
        name="Zazi 2023 Agent",
        instructions=instructions_2023,
        model="gpt-4o-mini"
)

zazi_2024_agent = Agent(
        name="Zazi 2024 Agent",
        instructions=instructions_2024,
        model="gpt-4o-mini"
)

zazi_2025_agent = Agent(
        name="Zazi 2025 Agent",
        instructions=instructions_2025,
        model="gpt-4o-mini"
)

Now let's test the first agent and see that it runs.

In [7]:
result = Runner.run_streamed(zazi_2023_agent, input="What is Zazi iZandi all about?")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Zazi iZandi is a literacy program implemented in South Africa aimed at improving children's reading skills. The program focuses on teaching small groups of children, usually consisting of seven, their letter sounds in a sequence based on the frequency of letter usage. 

Key features include:

- **Group Learning**: Children are taught in small, level-based groups, allowing for targeted instruction that matches their current literacy level.
  
- **Focus on Letter Sounds**: The curriculum emphasizes teaching the sounds of letters, which is foundational for reading and writing skills.

- **Use of Letter Tracker**: Teacher Assistants (TAs) utilize an official Letter Tracker that organizes letters according to their frequency, ensuring that children learn the most commonly used letters first. 

This approach is designed to make learning more effective and engaging, helping children to develop their literacy skills in a structured manner.

Okay, now let's pull in some data to give the agents some basic tools. Just going to set root path and then import some of the data loaders & preprocessors

In [8]:
import os
import sys
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import streamlit as st

# We need to set root directory so we can find the zz_data_process_23.py file. I should obviously move this into a better named directory, will do so in the future.
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)

#2023 Data
from zz_data_process_23 import process_zz_data_23
from data_loader import load_zazi_izandi_2023

#2024 Data
from zz_data_processing import process_zz_data_midline, process_zz_data_endline, grade1_df, gradeR_df
from data_loader import load_zazi_izandi_2024


Create functions to import 2023 and 2024 data.

In [9]:
    
def import_2023_results():
    # Import dataframes
    endline_df, sessions_df = load_zazi_izandi_2023()
    endline = process_zz_data_23(endline_df, sessions_df)
    return endline

In [10]:
def import_2024_results():
    baseline_df, midline_df, sessions_df, baseline2_df, endline_df, endline2_df = load_zazi_izandi_2024()
    
    # Create deep copies to ensure data independence between tabs
    baseline_df = baseline_df.copy()
    midline_df = midline_df.copy()
    sessions_df = sessions_df.copy()
    endline_df = endline_df.copy()

    midline, baseline = process_zz_data_midline(baseline_df, midline_df, sessions_df)
    endline = process_zz_data_endline(endline_df)
    grade1 = grade1_df(endline)
    gradeR = gradeR_df(endline)
    
    return endline

Okay, let's create a super simple tool.

In [11]:
@function_tool
def get_2023_number_of_children():
    """
    Get the number of children on the programme in 2023
    """
    endline_2023 = import_2023_results()
    number_of_children = len(endline_2023)
    return number_of_children

@function_tool
def get_2024_number_of_children():
    """
    Get the number of children on the programme in 2024
    """
    endline_2024 = import_2024_results()
    number_of_children = len(endline_2024)
    return number_of_children

Let's update the instructions to tell the LLM to use the tool to get the number of children if asked.

In [12]:
instructions_2023 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. If you need to know the number of children on the programme, you can use the get_2023_number_of_children function."
instructions_2024 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2024. {zz_background}. If you need to know the number of children on the programme, you can use the get_2024_number_of_children function."
instructions_2025 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2025. {zz_background}. If you need to know the number of children on the programme, you can use the get_2025_number_of_children function."


Now let's update the agents to have the tools we created.

In [13]:
zazi_2023_agent = Agent(
        name="Zazi 2023 Agent",
        instructions=instructions_2023,
        model="gpt-4o-mini",
        tools=[get_2023_number_of_children]
)

zazi_2024_agent = Agent(
        name="Zazi 2024 Agent",
        instructions=instructions_2024,
        model="gpt-4o-mini",
        tools=[get_2024_number_of_children]
)

In [14]:
question = "How many children were on the programme in 2023?"

In [15]:
with trace("Zazi 2023 Agent"):
    result = await Runner.run(zazi_2023_agent, question)

print(result.final_output)

2025-09-12 09:19:59.133 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 09:19:59.134 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
/Users/jimmckeown/Python/ZZ Data Site/venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In 2023, there were **1,896 children** enrolled in the Zazi iZandi literacy programme.


Below is a version of the 2023 agent streaming back the results.

In [16]:
result = Runner.run_streamed(zazi_2023_agent, input=question)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

2025-09-12 09:20:06.938 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/Users/jimmckeown/Python/ZZ Data Site/venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In 2023, there were **1,896 children** enrolled in the Zazi iZandi literacy programme.

# AGENTIC SOLUTION

Okay, now let's create a more agentic solution. We will convert the 2023 and 2024 agents into tools themselves. And have a supervisor AI that calls on them if needed.

Let's start with updating the 2023 and 2024 prompts a bit. So those agents know a bit more about their respective years. 

Now let's give the agents some more tools.

instructions_2023 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. 

In 2023, Zazi iZandi was piloted in 12 schools. 52 youth were hired to work with 1897 children. The Grade 1 children improved their Early Grade Reading Assessment (EGRA) scores from 24 to 47. The Grade R children improved their EGRA scores from 5 to 26.

If you need to know the number of children on the programme, you can use the get_2023_number_of_children function."


In [17]:
instructions_2023 = f"""You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. 
In 2023, Zazi iZandi was piloted in 12 schools. The pilot ran for 3 months, from August to October. 52 youth were hired to work with 1897 children. 

# RESULTS
 
The Grade 1 children improved their Early Grade Reading Assessment (EGRA) scores from 24 to 47.
The Grade 1 children improved the number of letters they knew from 13 to 21. 
The percent of Grade 1 children that reached the target Reading Benchmark increased to 74%.
The Grade R children improved their EGRA scores from 5 to 26.
The Grade R children improved the number of letters they knew from 3 to 12. 

#TOOLS
If you need to know the number of children on the programme, you can use the get_2023_number_of_children function."""


In [ ]:
instructions_2024 = f"""You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. 
In 2024, Zazi iZandi was piloted in 16 schools. 82 youth were hired to work with 3490 children. 

# RESULTS
 
The Grade 1 children improved their Early Grade Reading Assessment (EGRA) scores from 14 to 38.
The percent of Grade 1 children that reached the target Reading Benchmark increased from 13%to 53%.
The Grade R children improved their EGRA scores from 1 to 25.

#TOOLS
If you need to know the number of children on the programme, you can use the get_2024_number_of_children function.
"""

Now that we updated the instructions, let's rerun the agent code.

In [19]:
zazi_2023_agent = Agent(
        name="Zazi 2023 Agent",
        instructions=instructions_2023,
        model="gpt-4o-mini",
        tools=[get_2023_number_of_children]
)

zazi_2024_agent = Agent(
        name="Zazi 2024 Agent",
        instructions=instructions_2024,
        model="gpt-4o-mini",
        tools=[get_2024_number_of_children]
)

Okay, now let's set these agents to be tools. This could be confusing at first, but both functions and agents themselves can be tools.

In [22]:
tool_2023 = zazi_2023_agent.as_tool(tool_name="2023_researcher", tool_description="Provides information, data, and statistics about the Zazi iZandi 2023 programme.")
tool_2024 = zazi_2024_agent.as_tool(tool_name="2024_researcher", tool_description="Provides information, data, and statistics about the Zazi iZandi 2024 programme.")


Now let's create a supervisor that has the ability to request information from the other agent tools.

In [ ]:
instructions_supervisor = """You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme. 
{zz_background}.

If the user asks for information about the 2023 programme, you can use the zazi_2023_agent.
If the user asks for information about the 2024 programme, you can use the zazi_2024_agent.

If no year is specified, assume the user in inquiring about 2024.

"""



In [24]:
zazi_supervisor = Agent(
        name="Zazi Supervisor",
        instructions=instructions_supervisor,
        model="gpt-4o",
        tools=[tool_2023, tool_2024]
)

In [27]:
question = "How did the children perform in 2023?"

with trace("Zazi iZandi Supervisor Agent"):
    result = await Runner.run(zazi_supervisor, question)

print(result.final_output)

In 2023, the children participating in the Zazi iZandi literacy programme showed remarkable progress:

### Grade 1 Children
- **EGRA Scores** improved from 24 to 47.
- **Letters Known** increased from 13 to 21.
- **Reading Benchmark**: 74% reached the target benchmark.

### Grade R Children
- **EGRA Scores** improved from 5 to 26.
- **Letters Known** increased from 3 to 12.

These results highlight a substantial improvement in literacy skills for both Grade R and Grade 1 children. Let me know if you need further insights!


In [26]:
from IPython.display import Markdown, display
display(Markdown(result.final_output))

In 2024, the performance of children in the Zazi iZandi literacy programme showed significant improvement:

### Grade 1 Children:
- **EGRA Scores:** Increased from 14 to 38.
- **Reading Benchmark Achievement:** The proportion of children reaching the target rose from 13% to 53%.

### Grade R Children:
- **EGRA Scores:** Increased from 1 to 25.

These results highlight a strong positive impact on children's reading skills and the effectiveness of the teaching strategies employed. If you need more details, feel free to ask!

Let's improve our agent a bit by adding a request for it to contextualize the data for the user.

# Next Steps
1. Add more utility to the 2023 and 2024 tools.
2. Create a 2025 agent (w/ corresponding tools).
3. Add to the supervisor prompt so that it includes some context.
4. Add in some RAG so that the agent can reference some Q&A.
5. Add in a guardrail so that no personal information is returned to the user.
6. Add in a tool for the agent to email us if someone wants to connect. It should get their personal info.
7. Add in a tool to send us a push notification and/or email if the LLM cannot answer a question.